In [1]:
dicc = {}

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Inicializa el WebDriver
driver = webdriver.Chrome()

# Navega a la página de estadísticas
url = 'https://www.binance.com/en/copy-trading'
driver.get(url)

# Espera a que la página cargue completamente
time.sleep(5)

#### selecciona 90 dias en el contenedor
# Esperar a que el contenedor del menú desplegable sea visible y clickeable
dropdown_trigger = WebDriverWait(driver, 2).until(
    EC.element_to_be_clickable((By.CLASS_NAME, 'bn-select-trigger'))
)
dropdown_trigger.click()  # Abrir el menú desplegable

# Esperar a que las opciones del menú sean visibles
options_container = WebDriverWait(driver, 2).until(
    EC.visibility_of_element_located((By.CLASS_NAME, 'bn-select-overlay-options'))
)
# Seleccionar la opción "90 Days"
option_90_days = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'bn-select-option') and .//div[text()='90 Days']]"))
)
option_90_days.click()  # Hacer clic en la opción "90 Days"


In [3]:

def next_page():
    # siguiente pagina
    try:
        # Espera hasta que el botón Next esté presente y sea clickeable
        next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'bn-pagination-next'))
            )
        next_button.click()  
        time.sleep(3)
        return True
    except Exception as err:
        print(f'error: {err}')
        return False

def scrape_page():
    # Encuentra todos los elementos que coinciden con la clase específica
    todos = driver.find_elements(By.CLASS_NAME, 'grid')
    # Extrae el texto de cada elemento encontrado
    lines = todos[0].text.split('\nMock\nCopy\n')
    for linea in lines:
        api = False
        linea_separada=linea.split('\n')        
        if linea_separada[2] == 'API':
            del linea_separada[2]
            api = True
        dicc[linea_separada[0]]={'seguidores':linea_separada[1],
                                'api':api,
                                'tiempo':linea_separada[2],
                                'pnl':linea_separada[3],
                                'roi':linea_separada[5],
                                'aum':linea_separada[7],
                                'mdd':linea_separada[9],
                                'sharpe':linea_separada[11]
                                }

pagina = 1

#adelantar paginas    
#for i in range(1,45,1):
#    next_page()   
#    pagina = i+1

while True:
    print(f'Página: {pagina}')
    scrape_page()
    pagina = pagina + 1
    if not next_page():
        break

Página: 1
Página: 2
Página: 3
Página: 4
Página: 5
Página: 6
Página: 7
Página: 8
Página: 9
Página: 10
Página: 11
Página: 12
Página: 13
Página: 14
Página: 15
Página: 16
Página: 17
Página: 18
Página: 19
Página: 20
Página: 21
Página: 22
Página: 23
Página: 24
Página: 25
Página: 26
Página: 27
Página: 28
Página: 29
Página: 30
Página: 31
Página: 32
Página: 33
Página: 34
Página: 35
Página: 36
Página: 37
Página: 38
Página: 39
Página: 40
Página: 41
Página: 42
Página: 43
Página: 44
Página: 45
Página: 46
Página: 47
Página: 48
Página: 49
Página: 50
Página: 51
Página: 52
Página: 53
Página: 54
Página: 55
Página: 56
Página: 57
Página: 58
Página: 59
Página: 60
Página: 61
Página: 62
Página: 63
Página: 64
Página: 65
Página: 66
Página: 67
Página: 68
Página: 69
Página: 70
Página: 71
Página: 72
Página: 73
Página: 74
Página: 75
Página: 76
Página: 77
Página: 78
Página: 79
Página: 80
Página: 81
Página: 82
Página: 83
Página: 84
Página: 85
Página: 86
Página: 87
Página: 88
Página: 89
Página: 90
Página: 91
Página: 

KeyboardInterrupt: 

In [7]:
#guardar diccionario
import json
# Guardar el diccionario en un archivo JSON
with open('leader_copy_trading.json', 'w') as guardar:
    json.dump(dicc, guardar)

In [8]:
# Cargar el diccionario desde un archivo JSON
import json
with open('leader_copy_trading.json', 'r') as cargar:
    diccionario_cargado = json.load(cargar)
len(diccionario_cargado)

3862

In [46]:
diccionario_cargado['User-25060']

{'seguidores': '0 / 200',
 'api': False,
 'tiempo': '90 Days PNL',
 'pnl': '0.00',
 'ROI': '0.00%',
 'AUM': '2,000.00',
 '90 Days MDD': '0.00%',
 'Sharpe Ratio': '–'}

In [58]:
# INSERTAR EN BASE DE DATOS

import psycopg2

# Función para convertir cadenas a flotantes
def parse_float(value):
    if isinstance(value, str):
        # Manejar el caso del ROI por separado para dividir por 100
        if '%' in value:
            value = value.replace(',', '').replace('+', '').replace('%', '').strip()
            try:
                return float(value) / 100  # Convertir a decimal
            except ValueError:
                return None
        else:
            value = value.replace(',', '').replace('+', '').strip()
            try:
                return float(value)
            except ValueError:
                return None  # O manejar el error de otra manera según tus necesidades
    return value

# Configuración de conexión
connection = psycopg2.connect(
host = "localhost",
database = "bd_aprendizaje",
user = "soporte",
password = ""
)

# Crear un cursor
cur = connection.cursor()
cur.execute("truncate table leaders;")
try:
    for nombre, metrics in diccionario_cargado.items():
            print(metrics['roi'])
            cur.execute("""
            INSERT INTO leaders (nombre, 
                        seguidores, 
                        api, 
                        tiempo, 
                        pnl,                     
                        roi,
                        aum
                        mdd, 
                        sharpe)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                nombre,
                metrics['seguidores'],
                metrics['api'],
                metrics['tiempo'],
                parse_float(metrics['pnl']),
                parse_float(metrics['ROI']),
                parse_float(metrics['AUM']),
                parse_float(metrics['90 Days MDD']),
                parse_float(metrics['Sharpe Ratio'])
            ))
    print("Datos insertados correctamente.")
except Exception as err:
    print(f'error: {err}')
    connection.rollback()

# Confirmar los cambios y cerrar la conexión
connection.commit()
cur.close()
connection.close()



error: 'roi'


In [61]:
metrics

{'seguidores': '591 / 700',
 'api': True,
 'tiempo': '90 Days PNL',
 'pnl': '+62,231.86',
 'ROI': '+269.45%',
 'AUM': '2,611,793.47',
 '90 Days MDD': '27.46%',
 'Sharpe Ratio': '3.66'}